## General Set Up

This code used the phidl library to algorithmically code complex geometries and map those to .gds files for photolithography.

When appropriate, please comment liberally to ease understanding for non-fluent users.


In [171]:
# Standard library imports
import warnings

# Third-party library imports
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from scipy.spatial.distance import cdist


# PHIDL imports
import phidl
import phidl.device_layout as dl
import phidl.routing as pr
import phidl.geometry as pg
import phidl.path as pp
from phidl import Device, Layer, LayerSet, Port
from phidl import quickplot as qp
from phidl import quickplot2 as qp2

# Set options for quickplot2
phidl.set_quickplot_options(
    show_ports=True,
    show_subports=False,
    label_aliases=True,
    new_window=True,
    blocking=False,
    zoom_factor=1.2,
    interactive_zoom=True,
)

from phidl.geometry import import_gds

# Other third-party libraries
import gdspy
import bezier
from ipywidgets import interact, IntSlider

# Suppress warnings
warnings.filterwarnings("ignore")

# For interactive plots
%matplotlib qt

## Defining Relevant Functions

The functions and classes below are used for final device encoding latter in the code.

In [173]:
def clear_view():
    """
    Clears the current plot to avoid overlapping visualizations.
    """
    import matplotlib.pyplot as plt
    plt.close('all')  # Closes all open figures

In [174]:
def check_and_center_device_x(device, center_on_x_axis=False):
    """
    Checks if the device is centered on the x-axis and optionally centers it.

    Parameters:
    - device: The device to check (PHIDL Device or SmartDevice).
    - center_on_x_axis (bool): If True, moves the device to center it on the x-axis.

    Returns:
    - None
    """
    # Calculate the bounding box of the device
    bbox = device.bbox  # Bounding box: [[xmin, ymin], [xmax, ymax]]

    # Calculate the x center of the bounding box
    x_center = (bbox[0][0] + bbox[1][0]) / 2

    # Print the bounding box and x-center
    print("Bounding Box:")
    print(f"Xmin: {bbox[0][0]:.2f}, Xmax: {bbox[1][0]:.2f}")
    print(f"X Center: {x_center:.2f} (Should be close to 0)")

    # If requested, move the device to center it on the x-axis
    if center_on_x_axis:
        device.move((-x_center, 0))  # Shift by negative x-center value
        print(f"The device has been moved to center it on the x-axis.")

In [175]:
def get_norm(x,y):
    """
    calculates the norm between two vectors

    Parameters:
    - x - input vector 
    - y - input vector 

    Returns:
    - dn - norm 
    """
    dy=np.gradient(y)
    dx=np.gradient(x)
    norm=np.array([dy,-dx])
    dn=norm/np.sqrt(np.sum(norm**2,axis=0))
    return dn

In [176]:
def thicken_to_poly(x,y,thick,flip_norm=False):
    
    """
    used to thicken

    Parameters:
    - TODO

    Returns:
    - TODO
    """
    norm_x,norm_y=get_norm(x,y)
    x_in,y_in=x+thick/2.*norm_x,y+thick/2.*norm_y
    x_out,y_out=x-thick/2.*norm_x,y-thick/2.*norm_y

    x_in=np.hstack((x_in[:-1],x_in[0]))
    y_in=np.hstack((y_in[:-1],y_in[0]))
    x_out=np.hstack((x_out[:-1],x_out[0]))
    y_out=np.hstack((y_out[:-1],y_out[0]))

    x=np.hstack((x_in,x_out[::-1]))
    y=np.hstack((y_in,y_out[::-1])) #reverse order to close polygon
    return np.vstack((x,y))

In [177]:
def snap_between_poly(poly1,poly2,idx1,idx2,ang_1,ang_2,mag1,mag2,
                      thickness,n_bezier=200,n_pts_1=3,n_pts_2=3,
                      n_ondul=0,amp_ondul=0):
    """
    creates a connecting region (like a "snap" or "bridge") between two polygons (poly1 and poly2) with 
    fine control over various parameters such as thickness, curvature, and ondulation (wavy patterns)
    
    parameters: 
    - poly1, poly2: Coordinates of the two polygons to connect.
    - idx1, idx2: Indices in the polygons where the connection starts and ends.
    - ang_1, ang_2: Angles of the normals at the start (poly1) and end (poly2) of the connection.
    - mag1, mag2: Magnitudes (or lengths) of the connecting region at the start and end.
    - thickness: The thickness of the connecting region.
    - n_bezier: Number of points for the Bezier curve, determining its smoothness.
    - n_pts_1, n_pts_2: Number of points around idx1 and idx2 used for averaging the normals.
    - n_ondul, amp_ondul: Number and amplitude of ondulations (wavy distortions) applied to the connection.
    """
    span_1=np.arange(idx1-n_pts_1,idx1+n_pts_1+1)%(len(poly1))
    span_2=np.arange(idx2-n_pts_2,idx2+n_pts_2+1)%(len(poly2))
    cap_1=poly1[span_1.astype(int),:]
    cap_2=poly2[span_2.astype(int),:]

    norm_1=get_norm(cap_1[:,0],cap_1[:,1])[:,n_pts_1]#[n_pts_1,:]#middle of the cap, i.e our pt1 point
    norm_2=get_norm(cap_2[:,0],cap_2[:,1])[:,n_pts_2]#[n_pts_2,:]#middle of the cap, i.e our pt1 point

    pt1=poly1[idx1]
    pt2=poly2[idx2]

    n_bits=2

    ang_1=ang_1/180*np.pi
    ang_2=ang_2/180*np.pi
    bitx1=[pt1[0]]#+mag1/float(n_bits)*norm_1[0]
    bitx2=[pt2[0]]
    bity1=[pt1[1]]
    bity2=[pt2[1]]
    for n in range(n_bits):
        n1_x=norm_1[0]*np.cos(n*ang_1)+norm_1[1]*np.sin(n*ang_1)
        n1_y=norm_1[1]*np.cos(n*ang_1)-norm_1[0]*np.sin(n*ang_1)

        n2_x=norm_2[0]*np.cos(n*ang_2)+norm_2[1]*np.sin(n*ang_2)
        n2_y=norm_2[1]*np.cos(n*ang_2)-norm_2[0]*np.sin(n*ang_2)

        bitx1.append(bitx1[-1]+mag1/float(n_bits)*n1_x)
        bitx2.append(bitx2[-1]+mag2/float(n_bits)*n2_x)

        bity1.append(bity1[-1]+mag1/float(n_bits)*n1_y)
        bity2.append(bity2[-1]+mag2/float(n_bits)*n2_y)
    bitx2=bitx2[::-1]
    bity2=bity2[::-1]
    #bitx1=[pt1[0]+x*norm_1[0] for x in np.linspace(0,mag1,n_bits)]
    #bitx2=[pt2[0]+x*norm_2[0] for x in np.linspace(mag2,0,n_bits)]
    #bity1=[pt1[1]+x*norm_1[1] for x in np.linspace(0,mag1,n_bits)]
    #bity2=[pt2[1]+x*norm_2[1] for x in np.linspace(mag2,0,n_bits)]

    #turn into closed polygon
    nodes1 = np.asfortranarray([
         bitx1+bitx2,
         bity1+bity2])
    curve1 = bezier.Curve(nodes1, degree=2*n_bits+1)

    #upper_edge
    #n_bezier=25
    thick_bez=thickness
    x_bez,y_bez= curve1.evaluate_multi(np.linspace(0,1,n_bezier+2))
    x_bez=x_bez[1:-1]
    y_bez=y_bez[1:-1]
    norm_bez=get_norm(x_bez,y_bez)

    #make it wavey
    theta=np.linspace(0,n_ondul*np.pi,len(x_bez))
    x_bez+=norm_bez[0]*np.sin(theta)*amp_ondul #make the latch ondulated
    y_bez+=norm_bez[1]*np.sin(theta)*amp_ondul
    norm_bez=get_norm(x_bez,y_bez)
    ret=np.hstack((x_bez,y_bez))

    pt1=cap_1[0,:] #first pt1 of cap
    pt2=cap_2[-1,:]
    bez_ctrl0=np.vstack( (x_bez+thick_bez/2.*norm_bez[0],y_bez+thick_bez/2.*norm_bez[1]))
    curve=bezier.Curve(np.asfortranarray(np.hstack((pt1.reshape(2,1),bez_ctrl0,pt2.reshape(2,1)))),n_bezier+1)
    upper_edge=curve.evaluate_multi(np.linspace(0,1,200))

    #lower_edge
    pt1=cap_2[0,:] #first pt1 of cap
    pt2=cap_1[-1,:]
    bez_ctrl1=np.vstack( (x_bez-thick_bez/2.*norm_bez[0],y_bez-thick_bez/2.*norm_bez[1]))[:,::-1]
    curve=bezier.Curve(np.asfortranarray(np.hstack((pt1.reshape(2,1),bez_ctrl1,pt2.reshape(2,1)))),n_bezier+1)
    lower_edge=curve.evaluate_multi(np.linspace(0,1,200))
   #print(norm_bez[0])

    xy=np.vstack( (upper_edge.T,cap_2[::-1,:],lower_edge.T,cap_1[::-1,:]))

    #route a midline between both edges:
    curve=bezier.Curve(np.asfortranarray(np.hstack((cap_1[n_pts_1,:].reshape(2,1),np.vstack((x_bez,y_bez)),cap_2[n_pts_2,:].reshape(2,1)))),n_bezier+1)
    midline=curve.evaluate_multi(np.linspace(0,1,200)) #this is useful for setting the gold path

    if n_ondul!=0 and amp_ondul!=0:

        c=midline.copy()
        arc_length_t=np.cumsum(np.hstack((0,np.sqrt(np.diff(c[0])**2+np.diff(c[1])**2))))
        arc_length_t/=arc_length_t[-1]
        t=arc_length_t*n_ondul*np.pi
        th_ond=0.5*(np.tanh((t-t[int(0.2*len(c[0]))]))+np.tanh((-(t-t[-1]+t[int(0.2*len(c[0]))]))))
        shift=np.sin(t)*th_ond
        norm_x,norm_y=get_norm(c[0],c[1])
        cx,cy=c[0]+amp_ondul*norm_x*shift,c[1]+amp_ondul*norm_y*shift

        norm_ond_x,norm_ond_y=get_norm(cx,cy)
        upper_edge[0]=th_ond*(cx+norm_ond_x*thickness/2.)+(1-th_ond)*upper_edge[0] #only modify where ond is strng
        upper_edge[1]=th_ond*(cy+norm_ond_y*thickness/2.)+(1-th_ond)*upper_edge[1] #only modify where ond is strng


        low_idx=len(c[0])-np.arange(len(c[0]))-1
        lower_edge[0][low_idx]=th_ond*(cx-norm_ond_x*thickness/2.)+(1-th_ond)*lower_edge[0][low_idx]
        lower_edge[1][low_idx]=th_ond*(cy-norm_ond_y*thickness/2.)+(1-th_ond)*lower_edge[1][low_idx]


        xy=np.vstack( (upper_edge.T,cap_2[::-1,:],lower_edge.T,cap_1[::-1,:]))
        midline=np.vstack((cx,cy))

        mid,u,l=midline,upper_edge,lower_edge
        dist_l=np.sqrt(np.sum((mid.T-u.T)**2,axis=1))
        dist_u=np.sqrt(np.sum((mid.T-l.T[::-1,::])**2,axis=1))


        idx=np.asarray(np.where((np.abs(dist_l-dist_u)>0.1*thickness)&(np.abs(dist_l-dist_u)<thickness)))
        idx=idx[(idx>5)&(idx<=len(dist_l)-6)] #make sure we don't move the hooks (end points)

        mid_cent=mid.copy()
        mid_cent[:,idx]=(u[:,idx]+l[:,-idx-1])/2.
        midline=mid_cent

    return xy,midline

In [178]:
def draw_device(a_device):   
    a_device # is this needed? 
    polys = a_device.get_polygons()

    from PIL import Image, ImageDraw

        
    """
    This is messy. could be automated but I was lazy.
    I upscale the quality and cast it to pixels, because
    I need a good way to import it into a meshing software for simulation
    And I do image analysis to reconstruct the mesh..
 
    parameters: 
    - TODO
    """
    img = Image.new('1', (5000, 5000))
    drw = ImageDraw.Draw(img, '1')

    span_x,span_y=np.diff(a_device.get_bounding_box().T)[:]
    scale=np.max((span_x,span_y))/(5000)

    shift_x,shift_y=(a_device.get_bounding_box()/scale)[0,:]

    for p in polys:
        p=np.vstack((p,p[0]))
        #p[:,0]-=np.min(p[:,0])
        #p[:,1]-=np.min(p[:,1])

        p=p/(1.5*scale) #scale

        #dimx,dimy=int(np.max(p[:,0])),int(np.max(p[:,1]))
        p[:,0]-=shift_x
        p[:,1]-=shift_y

        drw.polygon(list(map(tuple,p)), (1))
    del drw

    return img,scale,shift_x,shift_y

In [179]:
def squircle(block_rad,x_c,y_c,p=3):
    squircle=Device()
    theta=np.linspace(0,2*np.pi,5000)
    r=block_rad/(np.abs(np.cos(theta))**p+np.abs(np.sin(theta))**p)**(1./p)
    x=r*np.cos(theta)
    y=r*np.sin(theta)
    #squircle.add_polygon(np.vstack((x,y)))
    return x,y

# rads=np.linspace(5500,8900,14)
# ps=np.linspace(3,7,14)
# for sq_rad,p in zip(rads,ps):
#     xs_sq=np.vstack((squircle(sq_rad,0,0,p=p))).T
#     f.add_polygon(thicken_to_poly(xs_sq[:,0],xs_sq[:,1],50))

In [180]:
# this function createst the subrings within a ring in which the gold wires are actually wired (so if scaling design up to use more electrodes, 
#in addition to increasing ring thickness would also need to account for increasing some of the variables here)
def chop_ring(ring_poly,n_long,n_lat):
    '''ringpoly is an N-2 ring polygon (wavey loop)'''
    if ring_poly.shape[0]==2:
        ring_poly=ring_poly.T
    l=ring_poly.shape[0]
    outer_xy=ring_poly[:l//2,:]
    inner_xy=ring_poly[l//2:,:][::-1,:]
    l=len(outer_xy)
    steps_long=l//n_long


    t=np.linspace(0,1,n_lat+3)[1:-1] #for latitude control
    if n_lat==2:
        t=np.linspace(0.4,0.6,n_lat+3)[1:-1]
    chopped_device=SmartDevice()
    smartblocks=[]

    for i in range(n_long):
        begin,stop=i*steps_long,((i+1)*steps_long+1)#+1 needed on the stop to stick
        idx=np.arange(begin,stop)%l
        inside,outside=inner_xy[idx,:],outer_xy[idx,:]
        for j in range(n_lat):

            out=outside+t[j]*(inside-outside)
            ins=outside+t[j+1]*(inside-outside)
            #mid_edge=0.5*(out[0,:]+ins[0,:])
            #mid_edge_2=0.5*(out[-1,:]+ins[-1,:])
            #block=np.vstack( (mid_edge,out,mid_edge_2,ins[::-1,:]))
            block=np.vstack((out,ins[::-1,:]))
            #hooks=[0,int(np.floor(len(out)/2))+1,len(out)+1,len(out)+1+int(np.ceil(len(ins)/2))]
            hooks=[0] # more hooks = overlapping hooks when tiling
            smartblocks.append(SmartBlock('ringblock_'+str(n_long)+'_'+str(n_lat),block,hooks))

    return smartblocks

In [181]:
def smart_latch(pol1,pol2,idx1,idx2,ang_1,ang_2,mag1,mag2,thickness,n_bezier,n_pts_1,n_pts_2,n_ondul=0,amp_ondul=0):
    polygon,midline=snap_between_poly(pol1,pol2,idx1,idx2,ang_1,ang_2,mag1,mag2,thickness,n_bezier,n_pts_1,n_pts_2,n_ondul,amp_ondul)
    hook_idx=[-n_pts_1-1,(len(polygon)-(2*(n_pts_1+n_pts_2+1)))//2+n_pts_2] #these are the middle point of each latch's caps
    return SmartBlock('latch',polygon,hook_idx=hook_idx,route_line=midline)

In [182]:
def smart_electrode(radius,x_pos,y_pos,resolution):
    t=np.linspace(0,2*np.pi,resolution)
    x=np.hstack((0,radius*np.cos(t)))+ x_pos #put a point in the center for hook
    y=np.hstack((0,radius*np.sin(t)))+ y_pos
    return SmartBlock('electrode',polygon=np.vstack((x,y)).T,hook_idx=[0])

In [183]:
class SmartBlock:
    def __init__(self,block_type,polygon,hook_idx,route_line=None):
        if not ((block_type in ['electrode','pad','latch','pad_to_open']) or ('ringblock' in block_type)) :
            raise ValueError('The allowed block types are electrode, pad, latch and ringblock and pad_to_open')
        self.block_type=block_type
        if polygon.shape[1]!=2:
            polygon=polygon.T
        self.polygon=polygon
        self.hook_idx=hook_idx
        self.normals=get_norm(polygon[:,0],polygon[:,1])[:,hook_idx].T
        self.route_line=route_line

In [184]:
## In def_route_gold_lines, can alter geometry and size of electrode pads by altering the following line of code:
## rect=pg.rectangle((300,4000)).movex(x-150).movey(y) (first two numbers are the size, and shifting by 150 centers the electrode pad
class SmartDevice(Device):
    def __init__(self):
        '''initializes everything the Device needs, and a few helper things such as the graph and the smart blocks
        that help with routing gold'''
        Device.__init__(self)

        self.graph=nx.Graph()
        self.smart_blocks=[]
        self.all_hook_idx=[]
        self.all_hook_pos=[]
    def add_smart_block(self,smart_block_instance,add_to_design=True,layer=0):
        '''adds a smart block. These blocks have helping hooks to accelerate gold-path routing
        Sometimes, these blocks are only helpers, or chopped-up larger structures. As such, it is desirable sometimes
        to only use these blocks as routing helpers, but use only use the large, non-chopped structure for the design
        to save on the number of polygons. add_to_design tells whether to add these blocks to the overall design.'''
        self.smart_blocks.append(smart_block_instance)
        if add_to_design:
            super(SmartDevice,self).add_polygon(smart_block_instance.polygon,layer=layer) #tells SmartDevice to go to parent (Device) and find addpoly
        self.all_hook_idx.append(smart_block_instance.hook_idx)
        self.all_hook_pos.append(smart_block_instance.polygon[smart_block_instance.hook_idx,:])

    def route_gold_lines(self,hook_tolerance=100,el_to_hook_tolerance=10,gold_thickness=5.):#microns
        def who_connects_who_closest(l1,l2,tol):
            l1=np.asarray(l1)
            l2=np.asarray(l2)
            xy1=self.linearized_hook_pos[l1]
            xy2=self.linearized_hook_pos[l2]

            dists=np.min(cdist(self.linearized_hook_pos[l1],self.linearized_hook_pos[l2]),axis=1)
            who=l2[np.argmin(cdist(self.linearized_hook_pos[l1],self.linearized_hook_pos[l2]),axis=1)]
            who=who[dists<tol] # distance has to be smaller than 100 microns (avoids pad-latches to connect to ring. later on..fix)
            return list(map(tuple,np.vstack( (l1[dists<tol],who)).T))
        def connect_closest(l1,l2,tol):
            l1=np.asarray(l1)
            l2=np.asarray(l2)
            xy1=self.linearized_hook_pos[l1]
            xy2=self.linearized_hook_pos[l2]

            dists=np.min(cdist(self.linearized_hook_pos[l1],self.linearized_hook_pos[l2]),axis=1)
            who=l2[np.argmin(cdist(self.linearized_hook_pos[l1],self.linearized_hook_pos[l2]),axis=1)]
             # distance has to be smaller than 100 microns (avoids pad-latches to connect to ring. later on..fix)
            return list(map(tuple,np.vstack( (l1,who)).T))
        def who_connects_who_radius(l1,l2):
            l1=np.asarray(l1)
            l2=np.asarray(l2)
            xy1=self.linearized_hook_pos[l1]
            xy2=self.linearized_hook_pos[l2]

            dists=cdist(self.linearized_hook_pos[l1],self.linearized_hook_pos[l2])
            kl1=l1[np.where(dists<2*hook_tolerance)[0]]
            kl2=l2[np.where(dists<2*hook_tolerance)[1]]
            return list(map(tuple,np.vstack((kl1,kl2)).T))
        def construct_path(hook_path,resolution):
            '''resolution of path in microns'''
            xy=[]
            cum_length_per_latch=[] #useful to thicken gold lines differently
            path_by_block=self.linearized_block_types[hook_path]
            for i in range(1,len(hook_path)):
                if path_by_block[i]==path_by_block[i-1]=='latch':
                    #begin_of_latch
                    cum_length_per_latch.append(   np.sum(np.sqrt(np.sum(np.diff(np.concatenate(xy),axis=0)**2,axis=1))   ))

                    block_number=self.linear_to_block_index[hook_path[i]]
                    midline=self.smart_blocks[block_number].route_line.T
                    if self.linearized_hook_indices[hook_path[i-1]]>self.linearized_hook_indices[hook_path[i]]: #path is flipped wrt route line
                        midline=midline[::-1,:]
                    xy.append(midline)
                    #end_of_latch
                    cum_length_per_latch.append(   np.sum(np.sqrt(np.sum(np.diff(np.concatenate(xy),axis=0)**2,axis=1))   ))

                else:
                    xy_end=self.linearized_hook_pos[hook_path[i]]
                    xy_start=self.linearized_hook_pos[hook_path[i-1]]
                    dist=np.sum((xy_end-xy_start)**2)
                    t=np.linspace(0,1,resolution)
                    xy.append((xy_start.reshape(2,1)+t*(xy_end.reshape(2,1)-xy_start.reshape(2,1))).T) #interpolate path with some res.

            x,y=np.concatenate(xy).T
            keep=np.where(np.abs(np.diff(x)+np.diff(y))>1e-2)[0]
            if len(x)-1 not in keep:
                keep=np.hstack((keep,len(x)-1)) #add back the pad, which is the last element
            x,y=x[keep],y[keep]
            return x,y,cum_length_per_latch

        def group_consecutives(vals, step=1):
            """Return list of consecutive lists of numbers from vals (number list)."""
            run = []
            result = [run]
            expect = None
            for v in vals:
                if (v == expect) or (expect is None):
                    run.append(v)
                else:
                    run = [v]
                    result.append(run)
                expect = v + step
            return result
        def thicken_path(x,y,start_w=gold_thickness/2.,stop_w=gold_thickness/2.,smooth=0.05,frac=0.2):
            '''smooth is the slope of tanh transition, frac is when to jump to stop_w'''
            t=np.arange(len(x))
            offset,div=len(x)*frac,len(x)*smooth
            transition = 0.5*(np.tanh( (t-offset)/div)+1)*(stop_w-start_w)+start_w
            xn,yn=get_norm(x,y)*transition
            xpol=np.hstack( (x+xn,(x-xn)[::-1]))
            ypol=np.hstack( (y+yn,(y-yn)[::-1]))
            return np.vstack((xpol,ypol)).T
        def thicken_path_modulate(x,y,amps,percs,steep):
            def tanh_smooth(amps,percs,steep,support):
                x=support
                percs=np.asarray(percs)*len(x)
                steep=np.asarray(steep)*len(x)
                module=np.ones(len(x))*amps[0] # starting thickness
                for idx,(a,p,s) in enumerate(zip(np.diff(amps),percs,steep)):

                    module+=0.5*(np.tanh( (x-p)/s)+1)*a
                return module
            transition=tanh_smooth(amps,percs,steep,np.arange(len(x)))
            xn,yn=get_norm(x,y)*transition
            xpol=np.hstack( (x+xn,(x-xn)[::-1]))
            ypol=np.hstack( (y+yn,(y-yn)[::-1]))
            return np.vstack((xpol,ypol)).T
        self.linearized_hook_indices=(np.cumsum(np.concatenate([np.ones(len(el)) for el in self.all_hook_idx]))-1).astype(int)
        self.linearized_block_types=np.concatenate([len(s.hook_idx)*(s.block_type,) for s in self.smart_blocks])
        self.linear_to_block_index=np.concatenate([i*np.ones(len(el.hook_idx)) for i,el in enumerate(self.smart_blocks)]).astype(int)
        self.linearized_hook_pos=np.concatenate(self.all_hook_pos)
        ##---We make a square grid from the blocks, and disallow diagonal walks, so there's no possible path-crossings on this grid
        self.connected_hooks=[]
        unique_block_types,appearance_order=np.unique(self.linearized_block_types,return_index=True)
        #check how many longitudinal and latitudinal blocks there are in each ring layer, from the smartblock name
        ring_layers=[list(map(int,el.split('_')[1:])) for el in unique_block_types[np.argsort(appearance_order)] if 'ringblock' in el]
        n_per_layer=np.prod(ring_layers,axis=1)
        #connect hooks into a square grid
        for idx,(long,lat) in enumerate(ring_layers):
            shift=np.hstack((0,np.cumsum(n_per_layer)))[idx] #each ring's starting hook index
            for i in range(long):
                for j in range(lat):
                    node=(i*lat+j)%n_per_layer[idx]+shift #ring arithmetic
                    if j!=lat-1:
                        node_down=(i*lat+j+1)%n_per_layer[idx]+shift
                        self.connected_hooks.append((node,node_down)) # every j hooks is on a new block at the other edge of the ring. this connection would be diagonal
                    node_side=((i+1)*lat+j)%n_per_layer[idx]+shift
                    self.connected_hooks.append( (node,node_side) )
        #grab indices by block-type
        els=[idx for idx,el in enumerate(self.linearized_block_types) if 'electrode' in el]
        latches=[idx for idx,el in enumerate(self.linearized_block_types) if 'latch' in el]
        pads=[idx for idx,el in enumerate(self.linearized_block_types) if 'pad' in el]
        block_hooks=[idx for idx,el in enumerate(self.linearized_block_types) if 'ringblock' in el]

        latch_inside_hooks=np.asarray([idx for idx,el in enumerate(self.linearized_block_types) if 'latch' in el])
        latch_inside_hooks=list(map(tuple,latch_inside_hooks.reshape(len(latch_inside_hooks)//2,2)))


        self.connected_hooks+=who_connects_who_closest(pads,latches,hook_tolerance)
        self.connected_hooks+=who_connects_who_closest(latches,block_hooks,hook_tolerance)
        self.connected_hooks+=who_connects_who_closest(els,block_hooks,hook_tolerance)
        self.connected_hooks+=who_connects_who_closest(els,latches,1) #electrodes should only connect to blocks here

        self.connected_hooks+=latch_inside_hooks
        source=int(1e9)
        drain=int(1e10)

        pool_sources=[(source,e) for e in els]
        pool_drain=[(drain,e) for e in pads]

        #All the pads are connected to a drain now, and all the electrodes to a source.
        #We can now run a node-disjoint flow algorithm between source and drain to find #electrode non-crossing paths.

        dists=np.sqrt(np.sum(np.diff(self.linearized_hook_pos[self.connected_hooks],axis=-2)**2,axis=-1))
        dists=np.hstack((dists.reshape(-1),np.ones(len(pool_drain)),np.ones(len(pool_sources))))
        pairs=self.connected_hooks+pool_drain+pool_sources
        self.graph=nx.from_edgelist(pairs)
        #add pool and drain edge pairs to also set them to capacity 1 below
        capacities=np.hstack( (np.asarray(pairs),dists.reshape(-1,1)+1e-1))

        dict_capa={(int(a[0]),int(a[1])):a[2] for a in capacities}
        nx.set_edge_attributes(self.graph, dict_capa, name='capacity')
        print('Finishing setting up the graph\n')
        all_paths=list(nx.algorithms.connectivity.disjoint_paths.node_disjoint_paths(self.graph,source,drain,flow_func=nx.flow.edmonds_karp))
        self.all_paths=[np.asarray(p)[np.asarray(p)<source] for p in all_paths]
        all_paths=d.all_paths
        #refine them to be the shortest
        for idx in range(len(self.all_paths)):
            avoid_nodes = np.concatenate([el for i,el in enumerate(self.all_paths) if i!=idx]) #except for current path, remove other gold lines from graph
            subgraph=self.graph.subgraph([el for el in self.graph.nodes if el not in avoid_nodes])
            short_path=nx.shortest_path(subgraph,all_paths[idx][0],all_paths[idx][-1],weight='capacity')
            short_path=[el for el in short_path if el<source]
            self.all_paths[idx]=short_path
        print('Finished routing the gold leads')
        
        #make sure paths are the shortest: take out all nodes of the n-1 other paths, and remake 'shortest route' between
        #electrode and pad shortest path.
        
        
        
        #Make all routed lines into polygons, and into a new device
        self.gold=Device()
        self.gold_paths=[]
        for pidx,p in enumerate(self.all_paths):
            x,y,cum_length_per_latch=construct_path(p,10)

            dx,dy=np.diff(x),np.diff(y)
            cost=(dx[1:]*dx[:-1]+dy[1:]*dy[:-1])/(np.sqrt(dx[1:]**2+dy[1:]**2)*np.sqrt(dx[:-1]**2+dy[:-1]**2))
            kink=np.where(cost<0.5)[0]
            kink=np.hstack((kink))#keep only smooth angles
            chunks=group_consecutives(kink+1)
            padded_chunks=list(map(lambda x: [x[0]-3]+x+[x[-1]+3],chunks)) #smooth out bezier
            curve=[]
            for idx,bit in enumerate(padded_chunks):
                if idx==0:
                    curve.append(np.vstack((x[:bit[0]+1],y[:bit[0]+1])).T)
                nodes = np.asfortranarray([x[bit],y[bit]])
                crv = bezier.Curve(nodes, degree=len(nodes[0])-1)

                x_bez,y_bez= crv.evaluate_multi(np.linspace(0,1,10))
                curve.append(np.vstack((x_bez,y_bez)).T)
                if idx!=len(padded_chunks)-1:
                    next_bit=padded_chunks[idx+1]
                    curve.append(np.vstack((x[bit[-1]:next_bit[0]+1],y[bit[-1]:next_bit[0]+1])).T)
                if idx==len(padded_chunks)-1:
                    curve.append(np.vstack((x[bit[-1]:],y[bit[-1]:])).T)

            self.gold_paths.append(np.concatenate(curve))
#
            curve_length_cum=np.cumsum(np.sqrt(np.sum(np.diff(d.gold_paths[pidx],axis=0)**2,axis=1)))

#             if len(cum_length_per_latch)==8:
#                 b1,b2=np.asarray(cum_length_per_latch)[[5,7]]#where to start thickening the gold #end of second and third latch
#             elif len(cum_length_per_latch)==6:
#                 b1,b2=np.asarray(cum_length_per_latch)[[3,5]]#end of first and second latch
#             elif len(cum_length_per_latch)==10:
#                 b1,b2=np.asarray(cum_length_per_latch)[[7,9]]#..

#             print(len(cum_length_per_latch))
#             print(b1,b2)
#             first_bout=np.argmin( np.abs(curve_length_cum-b1))/float(len(curve_length_cum)) #inner electrode have longer path
#             second_bout=np.argmin(np.abs(curve_length_cum-b2))/float(len(curve_length_cum))
#             third_bout = np.argmin(np.abs(curve_length_cum-cum_length_per_latch[-3]))/float(len(curve_length_cum))
#             fourth_bout = (np.argmin(np.abs(curve_length_cum-cum_length_per_latch[-2]))-20)/float(len(curve_length_cum))#beg of last latch

#             #print(np.argmin( np.abs(curve_length_cum-b1)),np.argmin(np.abs(curve_length_cum-b2)))
#             self.gold.add_polygon(thicken_path_modulate(np.concatenate(curve)[:,0],np.concatenate(curve)[:,1],
#                                                    amps=[1.5,5,8,4,2.5],percs=[first_bout,second_bout,third_bout,fourth_bout]
#                                                         ,steep=[0.001,0.001,0.001,0.005]))

            self.gold.add_polygon(thicken_path(x,y,
                                               start_w=gold_thickness/2.,stop_w=gold_thickness/2.,frac=0.2))
        for idx in els:
            self.gold.add_polygon(self.smart_blocks[self.linear_to_block_index[idx]].polygon)
        #for idx in pads:
        #    self.gold.add_polygon(self.smart_blocks[self.linear_to_block_index[idx]].polygon)
        pads=[idx for idx,el in enumerate(self.smart_blocks) if el.block_type=='pad']
        for idx,p in enumerate(pads):
            x,y=self.all_hook_pos[p][0]
            rect=pg.rectangle((100,2700)).movex(x-50).movey(y) #put rectangular pad
            self.gold<<rect
            #self.add_smart_block(SmartBlock('pad_to_open',np.asarray(rect.get_polygons())[0],[0]),add_to_design=False)
    def make_top_layer(self):
        '''This takes the polygons of pad_to_open smart blocks, and electrode smart blocks
        and operates and optimized boolean substraction on the first layer'''
        def quick_bool(A,B):
            holes=B.get_polygons()
            A_p=A.get_polygons()
            index_keeper=np.concatenate([np.ones(len(el))*i for i,el in enumerate(A_p)]).astype(int)
            flat_A_p=np.asarray(np.concatenate(A_p))
            aTree=cKDTree(flat_A_p)
            bTree=cKDTree(np.concatenate(holes))
            where=np.concatenate(bTree.query_ball_tree(aTree,r=50))
            all_bool_As=np.unique(index_keeper[where.astype(int)])
            temp_A=Device()
            for i in all_bool_As:
                temp_A.add_polygon(A_p[int(i)])
            bigpad_idx=np.argmax(np.asarray(list(map(lambda x: np.max(x[:,1]),A_p))))
            temp_A.add_polygon(A_p[bigpad_idx]) #add polygon with largest 'y' <- its the fat pad. it has too few points to be detected as a neighbor
            opened=pg.boolean(temp_A,B,operation='A-B')
            for i in range(len(A_p)):
                if i not in all_bool_As:
                    if i!=bigpad_idx: #dont add the large pad a second time
                        opened.add_polygon(A_p[i])
            return opened
        holes=Device()
        open_pads=[el.polygon for idx,el in enumerate(self.smart_blocks) if el.block_type=='pad_to_open']
        els=[el.polygon for idx,el in enumerate(self.smart_blocks) if el.block_type=='electrode']
        for o in open_pads:
            holes.add_polygon(open_pads)
        for e in els:
            holes.add_polygon(els)
        self.third_layer=quick_bool(self,holes)

In [185]:
# from scipy.spatial.distance import cdist
# import numpy as np
# import networkx as nx
# from phidl.device_layout import Device
# import phidl.geometry as pg
# from scipy.spatial import cKDTree

# # Helper function for connecting the closest hooks
# def who_connects_who_closest(l1, l2, tol, hook_positions):
#     l1 = np.asarray(l1)
#     l2 = np.asarray(l2)

#     if len(l1) == 0 or len(l2) == 0:
#         return []

#     xy1 = hook_positions[l1]
#     xy2 = hook_positions[l2]

#     dists = cdist(xy1, xy2)
#     min_dist = np.min(dists, axis=1)
#     closest_indices = np.argmin(dists, axis=1)

#     valid_connections = min_dist < tol
#     connected_pairs = list(zip(l1[valid_connections], l2[closest_indices[valid_connections]]))
#     return connected_pairs


# # Define the SmartDevice class
# class SmartDevice(Device):
#     def __init__(self):
#         super().__init__()
#         self.graph = nx.Graph()
#         self.smart_blocks = []
#         self.all_hook_idx = []
#         self.all_hook_pos = []

#     def add_smart_block(self, smart_block_instance, add_to_design=True, layer=0):
#         """
#         Adds a smart block with hooks to assist in routing gold lines.
#         """
#         self.smart_blocks.append(smart_block_instance)
#         if add_to_design:
#             self.add_polygon(smart_block_instance.polygon, layer=layer)
#         self.all_hook_idx.append(smart_block_instance.hook_idx)
#         self.all_hook_pos.append(smart_block_instance.polygon[smart_block_instance.hook_idx, :])

#     def route_gold_lines(self, hook_tolerance=100, el_to_hook_tolerance=10, gold_thickness=5.0):
#         """
#         Routes gold lines by connecting hooks based on proximity and routing logic.
#         """
#         # Flatten hook positions
#         self.linearized_hook_pos = np.concatenate(self.all_hook_pos)

#         # Identify different block types
#         pads = [idx for idx, block in enumerate(self.smart_blocks) if block.block_type == "pad"]
#         latches = [idx for idx, block in enumerate(self.smart_blocks) if block.block_type == "latch"]
#         block_hooks = [idx for idx, block in enumerate(self.smart_blocks) if block.block_type == "block_hook"]
#         electrodes = [idx for idx, block in enumerate(self.smart_blocks) if block.block_type == "electrode"]

#         print("Pads:", pads)
#         print("Latches:", latches)
#         print("Block Hooks:", block_hooks)
#         print("Electrodes:", electrodes)

#         # Connect hooks between pads, latches, and electrodes
#         self.connected_hooks = []
#         self.connected_hooks += who_connects_who_closest(pads, latches, hook_tolerance, self.linearized_hook_pos)
#         self.connected_hooks += who_connects_who_closest(latches, block_hooks, hook_tolerance, self.linearized_hook_pos)
#         self.connected_hooks += who_connects_who_closest(electrodes, block_hooks, hook_tolerance, self.linearized_hook_pos)
#         self.connected_hooks += who_connects_who_closest(electrodes, latches, el_to_hook_tolerance, self.linearized_hook_pos)

#         print("Connected Hooks:", self.connected_hooks)

#         # Define source and drain
#         source, drain = int(1e9), int(1e10)
#         pool_sources = [(source, e) for e in electrodes]
#         pool_drain = [(drain, p) for p in pads]
#         self.connected_hooks += pool_sources + pool_drain

#         # Add edges to the graph
#         for conn in self.connected_hooks:
#             self.graph.add_edge(conn[0], conn[1], capacity=1)

#         # Add fallback connections if no valid path exists
#         if not nx.has_path(self.graph, source, drain):
#             print("Warning: No path between source and drain. Adding fallback connections.")
#             for node in self.graph.nodes:
#                 if node != source and node != drain:
#                     self.graph.add_edge(source, node, capacity=1)
#                     self.graph.add_edge(node, drain, capacity=1)

#         # Compute node-disjoint paths
#         all_paths = list(nx.algorithms.connectivity.disjoint_paths.node_disjoint_paths(
#             self.graph, source, drain, flow_func=nx.flow.edmonds_karp
#         ))

#         self.all_paths = [np.asarray(p)[np.asarray(p) < source] for p in all_paths]

#         # Refine paths to ensure no overlaps
#         for idx, path in enumerate(self.all_paths):
#             avoid_nodes = np.concatenate([p for i, p in enumerate(self.all_paths) if i != idx])
#             subgraph = self.graph.subgraph([n for n in self.graph.nodes if n not in avoid_nodes])
#             shortest_path = nx.shortest_path(subgraph, path[0], path[-1], weight="capacity")
#             self.all_paths[idx] = [n for n in shortest_path if n < source]

#         print("Final Paths:", self.all_paths)

#         # Create polygons for gold wires
#         self.gold = Device()
#         for path in self.all_paths:
#             if len(path) > 1:
#                 x, y, _ = self.construct_path(self.linearized_hook_pos[path], resolution=10)
#                 if len(x) > 0:
#                     self.gold.add_polygon(self.thicken_path(x, y, start_w=gold_thickness / 2, stop_w=gold_thickness / 2))

#     def make_top_layer(self):
#         """
#         Creates the top layer by subtracting pad and electrode regions from the overall layout.
#         """
#         def quick_boolean(A, B):
#             holes = B.get_polygons()
#             A_polygons = A.get_polygons()
#             index_keeper = np.concatenate([np.ones(len(poly)) * i for i, poly in enumerate(A_polygons)]).astype(int)
#             flat_A = np.concatenate(A_polygons)
#             a_tree = cKDTree(flat_A)
#             b_tree = cKDTree(np.concatenate(holes))
#             where = np.concatenate(b_tree.query_ball_tree(a_tree, r=50))
#             all_bool_As = np.unique(index_keeper[where.astype(int)])
#             temp_A = Device()
#             for i in all_bool_As:
#                 temp_A.add_polygon(A_polygons[int(i)])
#             big_pad_idx = np.argmax([np.max(poly[:, 1]) for poly in A_polygons])
#             temp_A.add_polygon(A_polygons[big_pad_idx])  # Add the largest pad
#             opened = pg.boolean(temp_A, B, operation="A-B")
#             for i in range(len(A_polygons)):
#                 if i not in all_bool_As and i != big_pad_idx:
#                     opened.add_polygon(A_polygons[i])
#             return opened

#         holes = Device()
#         open_pads = [block.polygon for block in self.smart_blocks if block.block_type == "pad_to_open"]
#         electrodes = [block.polygon for block in self.smart_blocks if block.block_type == "electrode"]

#         for pad in open_pads:
#             holes.add_polygon(pad)
#         for electrode in electrodes:
#             holes.add_polygon(electrode)

#         self.top_layer = quick_boolean(self, holes)
#         print("Top layer created.")

#     @staticmethod
#     def construct_path(hook_positions, resolution=10):
#         if len(hook_positions) < 2:
#             return np.array([]), np.array([]), []

#         xy = []
#         for i in range(1, len(hook_positions)):
#             xy_start = hook_positions[i - 1]
#             xy_end = hook_positions[i]
#             t = np.linspace(0, 1, resolution)
#             xy_segment = (1 - t)[:, None] * xy_start + t[:, None] * xy_end
#             xy.append(xy_segment)
#         xy = np.concatenate(xy, axis=0)
#         x, y = xy[:, 0], xy[:, 1]
#         return x, y, None

#     @staticmethod
#     def thicken_path(x, y, start_w=2.5, stop_w=2.5, smooth=0.05, frac=0.2):
#         if len(x) < 2:
#             return np.array([])

#         t = np.arange(len(x))
#         offset, div = len(x) * frac, len(x) * smooth
#         transition = 0.5 * (np.tanh((t - offset) / div) + 1) * (stop_w - start_w) + start_w
#         dx, dy = np.diff(x), np.diff(y)
#         norms = np.sqrt(dx**2 + dy**2)
#         dx, dy = dx / norms, dy / norms
#         x_pol = np.hstack([x[:-1] + dy * transition[:-1], x[-1::-1] - dy[::-1] * transition[-1::-1]])
#         y_pol = np.hstack([y[:-1] - dx * transition[:-1], y[-1::-1] + dx[::-1] * transition[-1::-1]])
#         return np.vstack([x_pol, y_pol]).T



# Active Region of the Script


### Define Ring Design

In [187]:
n_loop=1536 #number of points along loop
n_ondulation=[1,1,1,1,1,1] #affects waviness of ring
n_latch=[96,96,96,48,48,48] #affects number of ribbons connecting rings to each other

rs=[6200,5234,4527,3198,2248,1655] #radii of each of the rings
r_mod=[0.03,0.03,0.03,0.03,0.05,0.05]
thicknesses=[280,40,40,40,40,40] #affects the thickness of each of the rings
polys=[]
mesh_thick=[8,8,8,8,8] #affects thickness of the latches

d=SmartDevice()

d_center_x = d.center[0]  # Get x-coordinate of device center
print(d_center_x)

ring_long_blocks=[1536,1536,1536,512,256,256] #these numbers shouldn't change, otherwise routing will get messed up
ring_long_lats=[48,9,6,2,3,2] #these numbers shouldn't change as well

ring_blox=SmartDevice()
for idx, (rad, thick) in enumerate(zip(rs, thicknesses)):
    r = rad + r_mod[idx] * rad * np.sin(np.linspace(0, n_ondulation[idx] * 2 * np.pi, n_loop))
    initial_rot = 30
    t = np.linspace(0, 2 * np.pi, n_loop) + initial_rot
    x, y = r * np.cos(t), r * np.sin(t)
    
    # Center the polygon
    x_shift = 2 * np.mean(x)
    y_shift = 2 * np.mean(y)
    x_centered_poly, y_centered_poly = x - x_shift, y - y_shift
    poly = thicken_to_poly(x_centered_poly, y_centered_poly, thick)
    polys.append(poly)
    d.add_polygon(poly)
    
    # Routing
    smart_ringblock_for_routing = chop_ring(poly, n_long=ring_long_blocks[idx], n_lat=ring_long_lats[idx])
    for s in smart_ringblock_for_routing:
        d.add_smart_block(s, add_to_design=False)
        ring_blox.add_smart_block(s, add_to_design=True)


qp2(d)

d_center_x = d.center[0]  # Get x-coordinate of device center
print(d_center_x)

## Add the armlets ##
## define parameters for armlets for each 4 levels
idx1=[0,0,0,0,0] #where the latch starts
idx2=[60,60,60,60,60] #where the latch ends
ang1=[130,10,10,10,10,10] #how quickly it turns away from normal from the departing
ang2=[110,10,10,10,10,10] #how quickly it turns into normal on the arrival
mag1=[50,50,50,50,50] #how far it shoots along the normal (controls the bending)
mag2=[350,150,250,150,150] # how far it shoots along the normal on arrival (controls the bending)
nbezier=[55,25,25,25,25] # how many points to resolve shape : can be played with to smoothen the end of latch
thickness=mesh_thick
npts1=[2,2,2,2,2] # how wide is the latch at interception
npts2=[2,2,2,2,10] # how wide is the latch at interception
## -------------- ##

##6 support winglets on the first layer##
s_idx1=[-3,-5,-7,3,5,7] #where the latch starts
s_idx2=[20,27,35,397,390,383] #where the latch ends
s_ang1=[10,50,10,50,50,10] #how quickly it turns away from normal from the departing
s_ang2=[10,10,50,20,-20,5] #how quickly it turns into normal on the arrival
s_mag1=[200,200,100,150,150,150] # how far it shoots along the normal (controls the bending)
s_mag2=[150,200,300,150,150,150] # how far it shoots along the normal on arrival (controls the bending)
# how many points to resolve shape : can be played with to smoothen the end of latch
winglet_thickness=8
##-------------- ##


latch_collection=[]

for l in range(len(thicknesses)-1):
    for n in range(n_latch[l]):

        inner=np.vstack( (polys[l][:,n_loop:].T,polys[l][:,n_loop:].T )) #from outer ring
        outer=np.vstack( (polys[l+1][:,:n_loop].T,polys[l+1][:,:n_loop].T) ) #to inner ring

        shift=int(n*n_loop/n_latch[l]) #how much to move-over along the ring
        smart_block=smart_latch(inner,outer,idx1=idx1[l]+shift,idx2=idx2[l]-shift,ang_1=ang1[l],
                             ang_2=ang2[l],mag1=mag1[l],mag2=mag2[l],n_bezier=nbezier[l],
                             thickness=thickness[l],n_pts_1=npts1[l],n_pts_2=npts2[l])

        #store latches for making more webs
        latch_collection.append([smart_block.polygon,n,l])
        d.add_smart_block(smart_block)

#iterative webbing
start_web_frac=[0.88,0.87,0.6,0.6,0.6]
end_web_frac=[0.09,0.16,0.4,0.4,0.4]
n_web = [0,0,2,2,2] #defines the number of webs, or ribbons interconnecting latches. for structural support
for lidx,j in enumerate(range(len(n_web))):
    for i in range(n_latch[lidx]):
        try:
            pol1=[el[0] for el in latch_collection if (el[1]==i) and (el[2]==j)][0]
            pol2=[el[0] for el in latch_collection if (el[1]==(i+1)%n_latch[lidx]) and (el[2]==lidx)][0]
            l1=len(pol1)//2
            l2=len(pol2)//2
            sub_pol=[]
            ##Webbing between latches
            for n in range(n_web[j]):
                frac1=np.linspace(start_web_frac[j],end_web_frac[j],n_web[j])[n]
                frac2=1-frac1 #the polygon runs inverse - we want them to match at the same 'height' along either latch,
                #such that the connecting latch is horizontal when the structure fully unfolds
                mag1,mag2,thickness=100,100,40
                if j>2:
                    mag1,mag2=30,30
                if j>2:
                    thickness=40
                pol,_=snap_between_poly(pol1[l1:,:],pol2[:l2,:],idx1=int(frac1*l1),idx2=int(frac2*l2),mag1=mag1,mag2=mag2,
                                       ang_1=10,ang_2=10,thickness=thickness,n_pts_1=3,n_pts_2=3,n_bezier=25)
                d.add_polygon(pol)
                sub_pol.append(pol)
                ##Webbing between webs
                if j<2: #skip for the innermost
                    if n>0:
                        subpol1=sub_pol[n-1]
                        subpol2=sub_pol[n]
                        sl1=sl2=len(subpol1)//2
                        if j==1: #next inner ring
                            if n<5:
                                for z in range(3):
                                    frac1=[0.2,0.5,0.8][z]
                                    frac2=1-np.linspace(0.3,0.85,3)[z]
                                    subpol,_=snap_between_poly(subpol2[sl1:,:],subpol1[:sl2,:],idx1=int(frac1*sl1),idx2=int(frac2*sl2),mag1=60,mag2=60,
                                                       ang_1=10,ang_2=10,thickness=3,n_pts_1=7,n_pts_2=7,n_bezier=25)
                                    d.add_polygon(subpol)

                            if n>=5:
                                for z in range(2):
                                    frac1=[0.2,0.8][z]
                                    frac2=1-np.linspace(0.3,0.9,2)[z]
                                    subpol,_=snap_between_poly(subpol2[sl1:,:],subpol1[:sl2,:],idx1=int(frac1*sl1),idx2=int(frac2*sl2),mag1=60,mag2=60,
                                                       ang_1=10,ang_2=10,thickness=3,n_pts_1=7,n_pts_2=7,n_bezier=25)
                                    d.add_polygon(subpol)
                        if j==0:
                            for z in range(3):
                                frac1=[0.2,0.5,0.8][z]
                                frac2=1-np.linspace(0.3,0.85,3)[z]
                                subpol,_=snap_between_poly(subpol2[sl1:,:],subpol1[:sl2,:],idx1=int(frac1*sl1),idx2=int(frac2*sl2),mag1=60,mag2=60,
                                                   ang_1=10,ang_2=10,thickness=3,n_pts_1=7,n_pts_2=7,n_bezier=25)
                                d.add_polygon(subpol)

        except:
            print(n_latch[lidx],lidx)
            continue
qp2(d)

0.0
-1.1284373675221104


### Make Electrodes

In [189]:
n_pads=96
n_el=96 #defines the number or electrodes
electrodes = SmartDevice() #defines "electrodes" -- a smart device (used to hold geometries) 
tmp=pg.Device() #defines a device called tmp(temporary)
#platinum=SmartDevice() #defines a smart device called platinum for the platinum electrodes 

for i in range(n_el):
    
    if i < 48:
        shifter = -1 - i % 2  # Alternates between last two radii (innermost and second innermost)
    else:
        shifter = -3  # Assigns to the third innermost ring

    # Assign radius and calculate radial position
    rad = rs[shifter]
    r = rad + r_mod[shifter] * rad * np.sin(np.linspace(0, n_ondulation[shifter] * 2 * np.pi, n_loop))

    # Initial rotation and angles
    initial_rot = 30  # Initial rotation offset in radians
    t = np.linspace(0, 2 * np.pi, n_loop) + initial_rot

    # Dynamic angular shift for alignment
    ang_shift = [7, -4, -3][shifter]
    ang_index = ang_shift + n_loop // (n_el // 2) * (i % 48)
    ang = t[ang_index % n_loop]  # Wrap around using modulo to ensure valid index
    the_r = r[ang_index % n_loop]

    # Calculate x, y coordinates
    x, y = the_r * np.cos(ang), the_r * np.sin(ang)

    # Apply additional shift for the second and third rings
    if shifter == -2:  # Second innermost ring
        x -= 30  # Adjust x-coordinate
        y += 0  # Adjust y-coordinate
    elif shifter == -3:  # Third innermost ring
        x -= 15  # Adjust x-coordinate
        y += 0  # Adjust y-coordinate

    # Center the coordinates
    x_centered, y_centered = x - x_shift, y - y_shift

    # Plot the electrode positions
    plt.plot(x_centered, y_centered, 'o')

    # Add the electrodes to the device "d"
    d.add_smart_block(smart_electrode(12.5, x_centered, y_centered, resolution=100), add_to_design=False)
    d << pg.circle(radius=15.5).movex(x_centered).movey(y_centered)

    # Add the electrodes to the device "electrodes"
    electrodes.add_smart_block(smart_electrode(12.5, x_centered, y_centered, resolution=100), add_to_design=False)
    electrodes << pg.circle(radius=15.5).movex(x_centered).movey(y_centered)


# check_and_center_device_x(d, center_on_x_axis=True)
# check_and_center_device_x(d, center_on_x_axis=False)

qp2(d)

In [190]:
electrodes.write_gds('platinum_electrodes.gds')

'platinum_electrodes.gds'

### Define Where Wires Enter the Ring and Where Wires Connect to Contact Pads

In [192]:
#Parameters

increase_IO_region = 4750 # increase or decrease this value to affect length of I/O region
outer_ring_edge = polys[0][:, :1536]  # Extract the top part of the ring edge from the polygons array.
out_points = outer_ring_edge[:, np.arange(n_el) + 676][:, ::-1]  # Select points from the ring, center along the vertical axis, and order them from left to right.


pad_distance = 11000 + increase_IO_region  # Distance from the top of the ring to the pads.
pad_gap = 165.25  # Spacing between adjacent pads.
turn_from_pad_gap = 2500  # Distance where lines start turning towards the pads.

increase_IO_region = 4750 # increase or decrease this value to affect length of I/O region
pad_distance = 11000 + increase_IO_region  # Vertical distance from the top of the ring to the pads
pad_gap = 165.25  # Horizontal spacing (pitch) between adjacent pads
turn_from_pad_gap = 2500  # Distance from the top of the ring where the lines start curving towards the pads

# Center `out_points` from the ring
# out_x_coords = out_points[0, :]
# out_x_center = np.mean(out_x_coords)  # Calculate the center of x-coordinates
# out_points[0, :] -= out_x_center  # Shift to center around x=0

# Recompute `my_virtual_polygon` to align with the centered `out_points`
my_virtual_polygon = np.vstack((
    out_points[0, :],  # Centered x-coordinates
    np.ones(n_pads) * (np.max(polys[0][1, :]) + pad_distance - turn_from_pad_gap)  # y-coordinates are shifted down
))

# Recompute `my_pad_points` to ensure the I/O wires are properly aligned
# pad_x_coords = np.arange(0, n_pads * pad_gap, pad_gap)
pad_x_coords = np.arange(0, n_pads * pad_gap, pad_gap) - n_pads // 2 * pad_gap  # Generate x-coordinates for pads
# pad_x_coords -= np.mean(pad_x_coords)  # Center x-coordinates around 0

my_pad_points = np.vstack((
    pad_x_coords,  # Centered x-coordinates
    np.ones(n_el) * (np.max(polys[0][1, :]) + pad_distance)  # y-coordinates offset downward
))

# **Center the pads on the x-axis and align them at y=0**
x_pad_center = np.mean(my_pad_points[0, :])  # Find current x-center
my_pad_points[0, :] -= x_pad_center  # Shift x-coordinates to center

# Create polygons for the pads
my_pad_polygons = [
    np.vstack((my_pad_points[:, el], my_pad_points[:, el] * 1.1)).T  # Extend each pad slightly for geometry computations
    for el in range(n_el)
]

# Center my_virtual_polygon on the x-axis
my_virtual_polygon[0, :] -= x_pad_center  # Shift virtual polygon x-coordinates
my_virtual_polygon[1, :] = np.max(polys[0][1, :]) + pad_distance - turn_from_pad_gap  # Maintain its y-offset

# Visualize the updated points to confirm alignment
plt.scatter(out_points[0, :], out_points[1, :], label='Ring Output Points')
plt.scatter(my_virtual_polygon[0, :], my_virtual_polygon[1, :], label='Virtual Polygon')
plt.scatter(my_pad_points[0, :], my_pad_points[1, :], label='Pads')
plt.legend()
plt.show()

# Connect the ring to the pads using updated positions
for n in range(n_el):
    # Create the connection from the ring to the intermediate virtual polygon
    sl1 = smart_latch(
        out_points.T,  # Updated start points
        my_virtual_polygon.T,  # Updated end points
        n, n, 0, 0, 0, 0, 5, 25, 2, 2
    )

    # Create the connection from the virtual polygon to pads
    flip_normal = 1
    if n == 0 or n == n_el - 1:  # Flip normals at the edges
        flip_normal = -1

    sl2 = smart_latch(
        my_virtual_polygon[:, ::-1].T,  # Start points (reversed virtual polygon)
        my_pad_points.T,  # Updated end points
        95 - n, n, 0, 0, flip_normal * 1000, flip_normal * 1000, 5, 25, 2, 2
    )

    # Plot the route lines for visualization
    plt.plot(*sl1.route_line, color='r', linewidth=1)  # Line from ring to virtual polygon
    plt.plot(*sl2.route_line, color='b', linewidth=1)  # Line from virtual polygon to pads

    # Combine the two route lines
    gold_line = np.hstack((sl1.route_line, sl2.route_line))
    sm = SmartBlock('latch', gold_line.T, hook_idx=[0, -1], route_line=gold_line)
    d.add_smart_block(sm, add_to_design=False)

    # Add the pad to the design
    d.add_smart_block(SmartBlock('pad', my_pad_polygons[n].T, hook_idx=[0]), add_to_design=False)
    qp2(d)


### Wire Routing

The below section of code takes the longest time to run, somewhere around 8 minutes 

In [194]:
d.route_gold_lines(hook_tolerance=100,el_to_hook_tolerance=10,gold_thickness=2)
d.make_top_layer()

# #add the wider gold lines
# d.gold=pg.boolean(d.gold,wider_gold_device,operation='A+B')


qp2(d.gold)

Finishing setting up the graph

Finished routing the gold leads


In [195]:
tmp = pg.Device()
tmp << d
tmp << d.gold

check_and_center_device_x(tmp, center_on_x_axis=True)
check_and_center_device_x(tmp, center_on_x_axis=False)
# Visualize the result
qp2(tmp)

Bounding Box:
Xmin: -7899.38, Xmax: 7899.38
X Center: 0.00 (Should be close to 0)
The device has been moved to center it on the x-axis.
Bounding Box:
Xmin: -7899.38, Xmax: 7899.38
X Center: 0.00 (Should be close to 0)


In [196]:
qp2(tmp)

In [197]:
print(tmp.center[0])

0.0


### Make Polygons

In [199]:

# Assuming `tmp`, `rs`, and `increase_IO_region` are already defined
d2 = pg.Device()
d2 = pg.copy(tmp)

# Center `d2` on the x-axis
check_and_center_device_x(d2, center_on_x_axis=True)

contactPad_width = 16400  # Defines total width of the device at the contact pad

# Create the tee near the contact pads extending into the I/O region
rect_trap = pg.tee(
    size=(contactPad_width, 2000),
    stub_size=(2600, 9000 + increase_IO_region),
    taper_type='fillet',
    layer=0
)
rect_trap.move((180, rs[0] + 8960 + increase_IO_region))  # Align near contact pad region

# Create the bottom tee, flipped 180 degrees
rect_trap_bottom = pg.tee(
    size=(4000, 0.01),
    stub_size=(2600, 8999.99 + increase_IO_region),
    taper_type='fillet',
    layer=0
).rotate(180)
rect_trap_bottom.move((180, rs[0] - 80))  # Align to match `rect_trap`

# Create a rectangle overlaying the contact pad region
rect3 = pg.rectangle((contactPad_width, 3000), layer=0)
rect3.move((-contactPad_width / 2, rs[0] + 10960 + increase_IO_region))  # Center the rectangle

# Align polygons on the x-axis before adding to the device
for poly in [rect_trap, rect_trap_bottom, rect3]:
    poly_center_x = poly.center[0]  # Calculate the x-center of the polygon
    poly.move((-poly_center_x, 0))  # Center the polygon on the x-axis

# Add aligned polygons to the device
d2 << rect_trap
d2 << rect_trap_bottom
d2 << rect3

# Center the entire device on the x-axis
d2_center_x = d2.center[0]  # Get x-coordinate of device center
print(d2_center_x)
d2.move((-d2_center_x, 0))  # Move the device so its center is at x = 0

# Clear the viewing window and visualize
clear_view()
qp2(d2)


Bounding Box:
Xmin: -7899.38, Xmax: 7899.38
X Center: 0.00 (Should be close to 0)
The device has been moved to center it on the x-axis.
0.0


In [223]:
d << rect_trap
d << rect_trap_bottom
d.third_layer << rect_trap
d.third_layer << rect_trap_bottom
d << rect3

DeviceReference (parent Device "rectangle", ports [], origin (0, 0), rotation 0, x_reflection False)

In [225]:
clear_view() # Clear viewing window before visualizing
qp2(d.third_layer)

### Used to create the support mesh 

In [229]:
fourth_layer = pg.Device()
dcopy = pg.copy(d.third_layer)
circle_bool = pg.circle(radius = rs[0] - 100).movex(0).movey(30)
circle_bool2 = pg.boolean(A = dcopy, B = circle_bool, operation = 'not', precision = 1e-3, num_divisions = [1,1], layer = 0)
qp2(circle_bool2)
fourth_layer << circle_bool2
qp2(fourth_layer)

### Write All Relevant Files For Download

In [322]:
d.write_gds('finalized_bottom_layer.gds')

'finalized_bottom_layer.gds'

In [323]:
d.gold.write_gds('finalized_second_gold_layer.gds')

'finalized_second_gold_layer.gds'

In [324]:
d.third_layer.write_gds('finalized_third_layer.gds')

'finalized_third_layer.gds'

In [325]:
fourth_layer.write_gds('finalized_fourth_layer.gds')

'finalized_fourth_layer.gds'

# Combine Multiple Devices into a Final Wafer

This section combines 8 completed multilayer devices into a single wafer design. 

#### The result is a multilayer GDSII file where the following is true:  
Layer 0 - common features that will be added to layer when making masks    
Layer 1 - Bottom passivating SU-8  
Layer 2 - Au features  
Layer 3 - Top passivating SU-8  
Layer 4 - Support structure SU-8 (patterned with thicker SU-8 to add rigidity to device)  


### Combine Layers

In [326]:
# Load GDS files
gds1 = import_gds("finalized_bottom_layer.gds")
gds2 = import_gds("finalized_second_gold_layer.gds")
gds3 = import_gds("finalized_third_layer.gds")
gds4 = import_gds("finalized_fourth_layer.gds")

# Create a new device to combine all layers
combined_device = Device("combined_device")

# Add polygons from each file to the combined device with unique layers
for gds, layer_id in zip([gds1, gds2, gds3, gds4], [1, 2, 3, 4]):
    for polygon in gds.get_polygons():
        combined_device.add_polygon(polygon, layer=layer_id)

# Save the combined device to a new GDS file
combined_device.write_gds("phidl_combined_layers_fixed.gds")

# Preview the combined design
qp2(combined_device)
print("Combined GDS saved as 'phidl_combined_layers_fixed.gds'")

Combined GDS saved as 'phidl_combined_layers_fixed.gds'


### Add box around device

In [327]:
# Assuming combined_device is your final combined device
D = combined_device  # Replace with your actual device

# Define spacing and layer for the box outline
spacing = 50  # Spacing in µm
outline_width = 5  # Width of the outline in µm
common_layer = Layer(0, 0)  # Define your desired layer

# Get the bounding box of the device directly
bbox = D.bbox  # Bounding box of the combined device
(xmin, ymin), (xmax, ymax) = bbox

# Expand the bounding box by the spacing
xmin -= spacing
ymin -= spacing
xmax += spacing
ymax += spacing

# Create the outer box
outer_box = pg.rectangle(size=(xmax - xmin, ymax - ymin), layer=common_layer).move((xmin, ymin))

# Create the inner box (to subtract from the outer box)
inner_box = pg.rectangle(size=(xmax - xmin - 2 * outline_width, ymax - ymin - 2 * outline_width), layer=common_layer).move((xmin + outline_width, ymin + outline_width))

# Subtract the inner box from the outer box to create a hollow outline
hollow_box = pg.boolean(A=outer_box, B=inner_box, operation="not", layer=common_layer)

# Add the hollow box outline to the combined device
D.add_ref(hollow_box)

# # Save the updated device with the hollow outline
# D.write_gds("device_with_hollow_outline.gds")
# qp2(D)
# print("Device with hollow outline saved as 'device_with_hollow_outline.gds'")

DeviceReference (parent Device "boolean", ports [], origin (0, 0), rotation 0, x_reflection False)

### 1x4 Array  of Devices

In [328]:
# Create a new device to hold the array
array_device = Device("4x1_device_array")

# Define the horizontal spacing for the devices (adjust for overlap)
spacing = xmax - xmin  # Device width

# Add 4 instances of the device D to the array
for i in range(4):  # 4 columns for the array
    device_instance = array_device.add_ref(D)  # Add a reference to D
    device_instance.move((i * spacing, 0))  # Position horizontally with adjusted spacing

### 2x4 Array  of Devices

In [329]:
# Create a new device to hold the final arrangement
final_device = Device("final_device_with_reflection")

# Add the original array to the final device
original_array_ref = final_device.add_ref(array_device)

# Determine the y-coordinate of the reflection axis
reflection_axis_y = array_device.bbox[0][1]  # The ymin of the bounding box of the array

# Reflect the array about the y-axis passing through the lowest feature
reflected_array_ref = final_device.add_ref(array_device)
reflected_array_ref.mirror(p1=(0, reflection_axis_y), p2=(1, reflection_axis_y))

# Save the final device to a GDS file
final_device.write_gds("final_device_with_reflection.gds")

# Quick plot to visualize
#qp2(final_device)
#print("Final device with reflected array saved as 'final_device_with_reflection.gds'")


'final_device_with_reflection.gds'

In [330]:
# Define the ring parameters
outer_diameter = 101_500  # Outer diameter of the ring in µm
ring_thickness = 1_500    # Thickness of the ring in µm
ring_layer = (0, 0)       # Layer for the ring

# Create the ring
outer_radius = outer_diameter / 2
inner_radius = outer_radius - ring_thickness
outer_circle = pg.circle(radius=outer_radius, layer=ring_layer)
inner_circle = pg.circle(radius=inner_radius, layer=ring_layer)
ring = pg.boolean(A=outer_circle, B=inner_circle, operation="not", layer=ring_layer)

# Calculate the current center of the `final_device`
(xmin, ymin), (xmax, ymax) = final_device.bbox
device_center_x = (xmin + xmax) / 2
device_center_y = (ymin + ymax) / 2

# Compute the shift to move the device center to (0, 0)
shift_x = -device_center_x
shift_y = -device_center_y

# Move the device to center it at (0, 0)
final_device.move((shift_x, shift_y))

# Add the ring to the final device
final_device.add_ref(ring)

# Save the updated device with the ring
final_device.write_gds("final_device_with_centered_ring.gds")

# Quick plot to visualize
qp2(final_device)
#print("Final device with centered ring saved as 'final_device_with_centered_ring.gds'")


In [ ]:
# Text content and settings
text_content = """Ventricular Mesh Version 1.0
Rayyan Nasser and Christopher Warren
Fu Lab, Bioengineering, Princeton University"""
text_layer = (0, 0)  # Text layer
text_size = 1000  # Text size in µm

# Create the text object
text_device = pg.text(text=text_content, size=text_size, layer=text_layer)

# Get the bounding box of the ring to calculate the horizontal center
ring_bbox = final_device.bbox
ring_center_x = (ring_bbox[0][0] + ring_bbox[1][0]) / 2  # Center of the ring

# Get the bounding box of the text to center it horizontally
text_bbox = text_device.bbox
text_width = text_bbox[1][0] - text_bbox[0][0]  # Width of the text
text_height = text_bbox[1][1] - text_bbox[0][1]  # Height of the text

# Calculate the position of the text
text_spacing = 2000  # Space between the text and the top of the ring in µm
text_position_x = ring_center_x - (text_width / 2)  # Center the text horizontally
text_position_y = ring_bbox[1][1] - text_spacing - text_height  # Place below the top of the ring

# Move the text to the calculated position
text_device.move(destination=(text_position_x, text_position_y))

# Add the text to the final device
final_device.add_ref(text_device)

# Save the updated device with the centered text
final_device.write_gds("final_device_with_centered_text.gds")

# # Quick plot to visualize
# qp2(final_device)
# #print("Final device with centered text saved as 'final_device_with_centered_text.gds'")

# Define text for each layer and corresponding layers
layer_texts = [
    ("Layer 1 - Bottom passivating SU-8", 1),
    ("Layer 2 - Au features", 2),
    ("Layer 3 - Top passivating SU-8", 3),
    ("Layer 4 - Support structure SU-8", 4),
]

layer_text_size = 1000  # Size of the text
text_spacing = 1200  # Spacing between lines in µm

# Get the bounding box of the previous text
previous_text_bbox = text_device.bbox
base_y_position = previous_text_bbox[0][1] - 2000  # Position below the previous text

# Loop through the layer texts and add them to the device
for i, (text, layer) in enumerate(layer_texts):
    # Create text for the current layer
    layer_text_device = pg.text(text=text, size=layer_text_size, layer=(layer, 0))
    
    # Calculate the position for the current text
    text_position_x = previous_text_bbox[0][0]  # Align horizontally with the previous text
    text_position_y = base_y_position - i * text_spacing  # Position each line below the previous one
    
    # Move the text to the calculated position
    layer_text_device.move(destination=(text_position_x, text_position_y))
    
    # Add the text to the final device
    final_device.add_ref(layer_text_device)

# Save the updated device with all layer-specific text
final_device.write_gds("final_device_with_layer_text.gds")

# Quick plot to visualize
qp2(final_device)
#print("Final device with layer-specific text saved as 'final_device_with_layer_text.gds'")


In [332]:
from phidl import Device
from phidl.geometry import litho_calipers

# Define the alignment marks GDS file
alignment_gds_file = "/Users/christopherwarren/Documents/Research/Fu Lab /AutoCAD/Cardiac Mesh/AlignmentMarks.gds"  # Update this to the correct file path

# Import the alignment marks
alignment_marks_device = import_gds(
    filename=alignment_gds_file,
    cellname=None,  # Automatically selects the topmost cell
    flatten=False
)

# Calculate the left side of the ring
ring_bbox = ring.bbox  # Get the bounding box of the ring
xmin, ymin = ring_bbox[0]  # Bottom-left corner
xmax, ymax = ring_bbox[1]  # Top-right corner

# Calculate the new position for the alignment marks
new_xleft = xmin + 10000  # 1000 µm inboard of the left side
new_xright = xmax - 10000  # 1000 µm inboard of the right side
ring_center_y = (ymin + ymax) / 2  # Center y-coordinate of the ring

# Move the alignment marks to the calculated position
alignment_marks_left = final_device.add_ref(alignment_marks_device)
alignment_marks_right = final_device.add_ref(alignment_marks_device)

alignment_marks_left.move((new_xleft - alignment_marks_device.x, ring_center_y - alignment_marks_device.y))
alignment_marks_right.move((new_xright - alignment_marks_device.x, ring_center_y - alignment_marks_device.y))

# Create lithography calipers
caliper_left = litho_calipers(notch_size=[2, 5], notch_spacing=2, num_notches=11, offset_per_notch=0.1, row_spacing=0, layer1=1, layer2=2)
caliper_right = litho_calipers(notch_size=[2, 5], notch_spacing=2, num_notches=11, offset_per_notch=0.1, row_spacing=0, layer1=1, layer2=2)

# Position the lithography calipers outboard of the alignment marks
align_bbox_left = alignment_marks_left.bbox  # Bounding box of left alignment mark
align_bbox_right = alignment_marks_right.bbox  # Bounding box of right alignment mark

# Calculate positions
caliper_left_x = align_bbox_left[0][0] - 5000  # 500 µm outboard of the left alignment mark
caliper_right_x = align_bbox_right[1][0] + 5000  # 500 µm outboard of the right alignment mark
caliper_y = (align_bbox_left[0][1] + align_bbox_left[1][1]) / 2  # Center y-coordinate

# Add and move the calipers to the calculated positions
caliper_left_ref = final_device.add_ref(caliper_left)
caliper_right_ref = final_device.add_ref(caliper_right)

caliper_left_ref.move((caliper_left_x - caliper_left_ref.x, caliper_y - caliper_left_ref.y))
caliper_right_ref.move((caliper_right_x - caliper_right_ref.x, caliper_y - caliper_right_ref.y))

phidl.geometry.litho_ruler(height=2, width=0.5, spacing=1.2, scale=[3, 1, 1, 1, 1, 2, 1, 1, 1, 1], num_marks=21, layer=0)

# Save the updated GDS file
final_device.write_gds("final_device_with_alignment_marks_and_calipers.gds")

# Plot the final device
qp2(final_device)
print("Final device with alignment marks and lithography calipers saved as 'final_device_with_alignment_marks_and_calipers.gds'")

Final device with alignment marks and lithography calipers saved as 'final_device_with_alignment_marks_and_calipers.gds'
